In [1]:
import pandas as pd
import numpy as np
import os
import glob
from datetime import datetime
from tqdm import tqdm

In [2]:
#신세계푸드 031440 / 신세계건설 034300 / 신세계 I&C 035510
# 광주신세계 037710 / (주식회사)신세계 004170 / (주식회사)이마트 139480 / 현대건설 000720 / CJ제일제당 097950
codes=['031440','034300','035510','037710','004170','139480','000720','097950']
forders = os.listdir('./data/종목데이터/20210628_20220709/')
print(forders)

['000720', '004170', '031440', '034300', '035510', '037710', '097950', '139480', 'df_000720.csv', 'df_004170.csv', 'df_031440.csv', 'df_034300.csv', 'df_035510.csv', 'df_037710.csv', 'df_097950.csv', 'df_139480.csv']


In [3]:
timelist = pd.date_range('09:01','15:20',freq='1min').time

In [5]:
len(timelist)

380

In [6]:
timelist.astype('str')[0][:2] +timelist.astype('str')[0][3:5]

'0901'

In [7]:
forders_list = []
for code in codes:
    forders_list.append(os.listdir('./data/종목데이터/20210628_20220709/{}'.format(code)))
    

In [8]:
len(forders_list)

8

In [9]:
# 매매거래 정지되었던 22년 4월 6,7,8일 전날 데이터로 채우기
# 035510,037710


nan_dates = ['_2022_04_06','_2022_04_07','_2022_04_08']
fill_date = [20220406,20220407,20220408]

for code in ['035510','037710']:
    i=0    
    for nan_date in nan_dates:
        df = pd.read_csv(r'./data/종목데이터/20210628_20220709/{}/{}{}.csv'.format(code,code, nan_date), index_col=0)
        df_0405 = pd.read_csv(r'./data/종목데이터/20210628_20220709/{}/{}_2022_04_05.csv'.format(code,code), index_col=0)

        df['시간'] = timelist.astype('str')[i][:2] +timelist.astype('str')[i][3:5]
        df['날짜'] = fill_date[i]
        df['종목명'] = df_0405['종목명']
        df['시가'] = df_0405['시가']
        df['고가'] = df_0405['고가']
        df['저가'] = df_0405['저가']
        df['종가'] = df_0405['종가']
        df['거래량'].fillna(0,inplace=True)
        df['거래대금'].fillna(0,inplace=True)
        df['누적체결매도수량'].fillna(0,inplace=True)
        df['대비부호'].fillna(0,inplace=True)

        df.to_csv('./data/종목데이터/20210628_20220709/{}/{}{}.csv'.format(code,code,nan_date))

        print('{}{}.csv'.format(code,nan_date))
        i+=1

035510_2022_04_06.csv


035510_2022_04_07.csv
035510_2022_04_08.csv
037710_2022_04_06.csv
037710_2022_04_07.csv
037710_2022_04_08.csv


In [11]:

i=0
# 파일 합치기
for code in codes:
    
    j=0   # 영업 정지일 찾기

    globals()['df_{}'.format(code)] = pd.DataFrame()

    for forder in forders_list[i]:

        df = pd.read_csv('./data/종목데이터/20210628_20220709/{}/{}'.format(code,forder), index_col=0, encoding='utf-8')
        
        if len(df) > 0 :
                        
            date = df[df['날짜'].notnull()]['날짜'][0]
            codenm= df[df['종목명'].notnull()]['종목명'][0]
            df.insert(0,'Datetime', df['날짜'].astype('str') + df['시간'].astype('str'))

            df['Datetime'] = df['Datetime'].apply(lambda _ : datetime.strptime(_,'%Y%m%d%H%M'))

            # 15시20분 이후 데이터 자르기
            t=0
            for time_ in df['Datetime'].apply(lambda _ : _.time()):
                if time_ > datetime.strptime('1520','%H%M').time():
                    df.drop(index=[t],axis=1,inplace=True)
                    t+=1
                else:
                    t+=1
            # 9시~3시20분 1분 타임라인 리스트
            timelist = pd.date_range('09:01','15:20',freq='1min').time.astype('str')

            # 빠진 시간
            abtimes =  list(set(timelist)-set(df['Datetime'].apply(lambda _ : _.time().strftime('%H:%M:%S'))))

            datetimes = []
            for ab in abtimes:
                datetimes.append(datetime.strptime(date.astype('str')+ ab,'%Y%m%d%H:%M:%S'))
                
            testdf = pd.DataFrame(columns=df.columns)
            testdf['Datetime'] = datetimes


            df = pd.concat([df,testdf])

            df.sort_values(by='Datetime', ascending=True, inplace=True)

            df = df[['Datetime','시가','고가','저가','종가','거래량','거래대금']]

            # df['시간'] = timelist
            
            # df['날짜'].fillna(value=date, inplace=True)
            # df['종목명'].fillna(value=codenm, inplace=True)

            df['시가'].ffill(inplace=True)
            df['고가'].ffill(inplace=True)
            df['저가'].ffill(inplace=True)
            df['종가'].ffill(inplace=True)

            
            df['거래량'].fillna(value=0, inplace=True)
            df['거래대금'].fillna(value=0, inplace=True)
            # df['대비부호'].fillna(value=0, inplace=True)

            globals()['df_{}'.format(code)] = pd.concat([globals()['df_{}'.format(code)], df])


        if len(df) ==0 :
            j+=1
            
    globals()['df_{}'.format(code)].reset_index(inplace=True,drop=True)

    print(i,code," : ",j)
    i+=1

0 031440  :  123
1 034300  :  123
2 035510  :  123
3 037710  :  123
4 004170  :  123
5 139480  :  123
6 000720  :  123
7 097950  :  123


In [12]:
df_000720.iloc[:380,:]

,Datetime,시가,고가,저가,종가,거래량,거래대금
0,2021-06-28 09:01:00,54973.0,55070.0,54681.0,54973.0,49076,2696800000
1,2021-06-28 09:02:00,54973.0,55168.0,54876.0,55070.0,21315,1172830000
2,2021-06-28 09:03:00,55070.0,55168.0,54778.0,54778.0,25534,1404920000
3,2021-06-28 09:04:00,54876.0,54876.0,54486.0,54681.0,20114,1099340000
4,2021-06-28 09:05:00,54778.0,54876.0,54584.0,54876.0,15079,825440000
...,...,...,...,...,...,...,...
375,2021-06-28 15:16:00,56627.0,56724.0,56627.0,56724.0,3536,200330000
376,2021-06-28 15:17:00,56627.0,56724.0,56627.0,56724.0,4459,252600000
377,2021-06-28 15:18:00,56724.0,56724.0,56627.0,56627.0,2633,149190000
378,2021-06-28 15:19:00,56724.0,56724.0,56627.0,56627.0,3348,189730000


In [13]:
for code in codes:
    
    globals()['df_{}'.format(code)].drop_duplicates(['Datetime'], keep='first', inplace=True, ignore_index=True)

    globals()['df_{}'.format(code)]['시가'].ffill(inplace=True)
    globals()['df_{}'.format(code)]['고가'].ffill(inplace=True)
    globals()['df_{}'.format(code)]['저가'].ffill(inplace=True)
    globals()['df_{}'.format(code)]['종가'].ffill(inplace=True)
    globals()['df_{}'.format(code)].to_csv('./data/종목데이터/20210628_20220709/df_{}.csv'.format(code),index=False)

In [14]:
codes

['031440',
 '034300',
 '035510',
 '037710',
 '004170',
 '139480',
 '000720',
 '097950']

In [15]:
df_037710.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96900 entries, 0 to 96899
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Datetime  96900 non-null  datetime64[ns]
 1   시가        96900 non-null  float64       
 2   고가        96900 non-null  float64       
 3   저가        96900 non-null  float64       
 4   종가        96900 non-null  float64       
 5   거래량       96900 non-null  int64         
 6   거래대금      96900 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(2)
memory usage: 5.2 MB


In [16]:
df_000720[df_000720['시가'].isnull()]['시가']

Series([], Name: 시가, dtype: float64)

In [17]:
df_000720.iloc[36860:,:]

,Datetime,시가,고가,저가,종가,거래량,거래대금
36860,2021-11-18 09:01:00,49150.0,49150.0,49100.0,49150.0,0,0
36861,2021-11-18 09:02:00,49150.0,49150.0,49100.0,49150.0,0,0
36862,2021-11-18 09:03:00,49150.0,49150.0,49100.0,49150.0,0,0
36863,2021-11-18 09:04:00,49150.0,49150.0,49100.0,49150.0,0,0
36864,2021-11-18 09:05:00,49150.0,49150.0,49100.0,49150.0,0,0
...,...,...,...,...,...,...,...
96895,2022-07-08 15:16:00,39650.0,39700.0,39650.0,39700.0,1385,54980000
96896,2022-07-08 15:17:00,39700.0,39700.0,39650.0,39700.0,553,21930000
96897,2022-07-08 15:18:00,39700.0,39700.0,39600.0,39650.0,2702,107150000
96898,2022-07-08 15:19:00,39650.0,39650.0,39600.0,39600.0,580,22970000
